In [ ]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys')
import run_with_config
importlib.reload(run_with_config)
from run_with_config import run_with_config


In [ ]:
#####

import market_data
importlib.reload(market_data)
from market_data import MarketDataManager

######

import fundamental_data
importlib.reload(fundamental_data)
from fundamental_data import run_pipeline

######

import tech_analysis
importlib.reload(tech_analysis)
from tech_analysis import TechAnalysisPipeline


######

import news_pipeline
importlib.reload(news_pipeline)
from news_pipeline import NewsPipeline

######

import data_integration
importlib.reload(data_integration)
from data_integration import DataIntegrator

######

import bonds_processor
importlib.reload(bonds_processor)
from bonds_processor import run_pipeline_bonds

## Globals

In [ ]:
token = "t.GVEetX09IVL9qECDUhjThfpg5TrE2Oxw9cUnAIOxcHh35M2sv1YMy9LBfkt73F3WlV_Ik08etZOVrUNI8StLBQ"
kursach_directory_path = "/Users/aeshef/Documents/GitHub/kursach"
your_directory_path = "/Users/aeshef/Documents/GitHub/kursach/data"

tickers = pd.read_csv(f"{your_directory_path}/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2024-01-01"
end_date="2025-04-15"
timeframe="1h"

YOUR_API_ID = 28994010
YOUR_API_HASH = "6e7a57fdfe1a10b0a3434104b42badf2"

overall_df = '/Users/aeshef/Documents/GitHub/kursach/data/df.csv'

## Initial parser

In [ ]:
# import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler('market_data.log'),
#         logging.StreamHandler()
#     ]
# )

# # Инициализация менеджера
# manager = MarketDataManager(
#     token=token,
#     base_directory=your_directory_path
# )

# # Получение данных
# data = manager.run_pipeline(
#     tickers=tickers_list,
#     start_date="2024-01-01",
#     end_date="2025-04-22"
# )

## Fundamental

In [ ]:
run_pipeline(ticker_list=tickers_list, base_path='./smartlab_data')

## Tech

In [ ]:
# pipeline = TechAnalysisPipeline(base_dir="/Users/aeshef/Documents/GitHub/kursach")
# pipeline.run_pipeline()

## News

In [ ]:
# run_with_config(
#     script_path="/Users/aeshef/Documents/GitHub/kursach/pys/news_pipeline.py",
#     class_or_function="NewsPipeline",
#     method="run_pipeline",
#     base_dir=kursach_directory_path,
#     config_filename="news_pipeline_config.json",
    
#     tickers=tickers_list,
#     collect_telegram=True,
#     telegram_api_id=YOUR_API_ID,
#     telegram_api_hash=YOUR_API_HASH,
#     telegram_channel="cbrstocks",
#     telegram_limit=10000,
#     start_date=start_date,
#     end_date=end_date,
#     use_cached_telegram=True
# )

In [ ]:
# integrator = DataIntegrator(base_path='/Users/aeshef/Documents/GitHub/kursach/data/processed_data', nan_fill_method='zero')
# combined_data = integrator.run_pipeline(tickers=tickers_list, output_path = overall_df)

In [ ]:
# Простой ежедневный запуск с настройками по умолчанию
bonds_results = run_pipeline_bonds(
    bonds_dir="/Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/csv",
    visualize=True,
    select_top_n=5,
    comparison_with_previous=True,  # Явно указываем параметр
    save_results=True,              # Явно указываем параметр
    use_latest_snapshot_only=True,
    output_dir="/Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/results"
)

# Проверяем результаты
print(f"Получено {len(bonds_results['top_bonds'])} облигаций для портфеля")


In [ ]:
# # Или запуск с настройками для конкретной рыночной ситуации
# if "ожидается снижение ключевой ставки":
#     daily_bonds = run_pipeline_bonds(
#         # Ожидая снижение ставки, увеличиваем вес доходности
#         score_weights={
#             'yield': 0.7,           # Больше вес на доходность
#             'duration': 0.2,        # Меньше вес на дюрацию
#             'tax_benefit': 0.05,    # Меньше вес на налоговую льготу
#             'price_discount': 0.05  # Меньше вес на скидку
#         },
#         market_condition='high_rate',
#     )


<!-- я делаю курсовой проект. я придерживаюсь этого плана:
# ---  
## 1. Слой Сбора и Агрегации Данных

### 1.1. Рыночные данные
- Источники API: Yahoo Finance, Alpha Vantage для акций. Для облигаций/ОФЗ — выгрузки или ручная подготовка, если нет автосбора.
- Загрузка исторических котировок всех бумаг, которые потенциально могут войти в портфель (акции + risk-free/фонды).
- Сохранение единой “сырых” данных (CSV/Parquet).

### 1.2. Фундаментальные данные
- Используются только последние доступные отчёты по каждой бумаге (например, за 2023–24, чаще нет).
- Допускается ручной ввод/подготовка csv коллегой (дефолтно — только 1 snapshot на тикер).
- Промышленный подход: между апдейтами отчётов — “forward fill” фундаментальных коэффициентов; при вычислениях делается анализ только cross-sectional (по состоянию на одну дату между бумагами).
- Стандартизация (z-score) коэффициентов для последующего скоринга.

### 1.3. Новостной/сентимент-поток
- Автоматизация выгрузки новостей (RSS, Twitter, базовые каналы для акций и risk-free).
- Анализ новостного сентимента (проще — ручная классификация “важные макро/кризисы” для risk-free бумаг).
- Сохранение текстовых и сентиментных фич в отдельные поля.

---

## 2. Слой Предобработки и Признаков

### 2.1. Генерация признаков для акций и risk-free
- Чистка рядов; заполнение пропусков.
- Генерация технических индикаторов (SMA, EMA, RSI, MACD, Bollinger Bands, паттерны — минимум сигналы типа "пересечение SMA").
- Для risk-free инструментов — расчет доходностей/дюраций “на сейчас”, если есть хотя бы скриншот доходности.
- Для всех сегментов — доходности, волатильности, ковариации.

### 2.2. Работа с фундаменталом (ограничена snapshot-режимом!)
- Для единственного года: сравнение бумаг между собой — расчёт z-score фундаментальных метрик на дату.
- Проведение статического ранжирования (квантили, top/bottom группы).
- Генерация сводного композитного score для отбора universe.

### 2.3. Интеграция сентимента и фундаментала
- Корректировка статичного funda-score с учётом свежих новостных сигналов (например, большой рынок негатив — понизить вес funda-score).
- Для risk-free — влияние только “макро”-новостей и текущей ставочной среды.

---

## 3. Модуль анализа и сигналов

### 3.1. Генерация торговых сигналов
- Акции: Buy/Hold/Sell по композитному score (технические + функдаментал + новости).
- Для risk-free: всегда Hold/Buy (core allocation), исключения — при негативных макро/ставках.

### 3.2. Кросс-секционный квантильный отбор
- “Shortlist” бумаг для портфеля: top 30–50% по funda-score, приоритизированных техническими индикаторами.
- Для risk-free инструментов можно просто фиксировать веса или задавать диапазон для оптимизации.

---  

## 4. Оптимизация Портфеля

### 4.1. Модель Марковица (Mean-Variance)
- Оптимизация среди shortlist бумаг (только Buy/Hold).
- Явно фиксируем вес на risk-free (“core allocation”, например 25–35%).
- Остальное распределяем по акциям оптимально по риск-доходности (mean-variance).

### 4.2. Black-Litterman (опционально)
- Возможность внести “view” на основе ранга funda-score и новостных сигналов.
- Коррекция “базовых” рыночных весов с помощью своих score.

---

## 5. Интеграция, отчётность и автоматизация

### 5.1. Финальные сигналы и обработка портфеля
- Взвешенная агрегация сигналов (funda-score × техника × новости).
- Единый пайплайн: загрузка — признаки — сигналы — оптимизация — результат.
- Пайплайны собраны как большие скрипты-автоматизаторы для (квази)ежедневного обновления.

### 5.2. Интерфейс пользователя
- Просмотр результатов
по кнопке (CLI/Jupyter/Telegram-бот).
- Возможность быстро изменить параметры universe/core allocation.

---

## 6. Бэктестинг и визуализация

### 6.1. Бэктест
- Проверка стратегии на годичном горизонте (по доступным данным).
- Особое внимание: для фундаментала — тестирование, как бы работали “портфели top vs bottom квантилей по snapshot”.
- Акцент на анализ разницы “просто держать весь рынок” vs. “отбирать по фунду/score”.

### 6.2. Визуализация
- Графики динамики портфеля, групп, эффективная граница.
- Визуализация сравнения групп, влияния сигналов.

---

## 7. Автоматизация и пайплайны

- Все ключевые этапы реализованы как отдельные большие py-скрипты (классы: DataCollector, FeatureEngineer, SignalGenerator, PortfolioOptimizer, Backtester, Visualizer).
- Каждый пайплайн сохраняет промежуточные дампы для отладки.
- Финальный мастер-пайплайн (“одна кнопка”).
- Делегирование ручных элементов коллегой: подготовка funda/csv, ручная разметка событий/новостей, первичный запуск функций.

---

## 8. Особенности и работа с мало меняющимися фундаментальными данными

- Прямо указывается в отчёте: фундаментальные признаки “медленные”, обновляются редко, анализ кросс-секционный: используется только сравнение между бумагами на одну дату, либо квантилизация и ранжирование при каждом отчёте.
- Расширение базы данных даже на 2–3 года (если возможно) — значительно повысит качество анализа и возможности rolling backtest.
- Все краткосрочные сигналы (техникa, новости) агрегируются для “ускорения” pipeline решений между публикациями отчетов.

---

## 9. Как преподнести в работе/отчёте

> Фундаментальные данные используются в cross-sectional анализе и квантилизации портфелей из-за своей низкой частоты обновления и ограниченной истории. Это соответствует практике индустрии для развивающихся рынков/ограниченных источников. При появлении новых отчетов pipeline автоматически обновляет ранжирование бумаг и состава портфеля. Краткосрочные решения строятся преимущественно на технических и новостных сигналах.

---

## Если понадобится — скину шаблоны py-файлов и классов по этому плану.


на данный момент я нахожусь на таком этапе: 
1) структура проекта
/Users/aeshef/Documents/GitHub/kursach
(├── df.csv - обьединенный датасет с 384 фичами для 17 тикеров за 2024-2025 год
├── figi.txt
├── news_analysis_summary.txt
├── processed_data
│   ├── AFKS
│   │   ├── AFKS.parquet
│   │   ├── AFKS_all_news_processed.csv
│   │   ├── AFKS_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── AFKS_daily_events.csv
│   │   │   ├── AFKS_daily_sentiment.csv
│   │   │   ├── AFKS_event_distribution.png
│   │   │   ├── AFKS_ml_features.csv
│   │   │   ├── AFKS_news_with_events.csv
│   │   │   ├── AFKS_news_with_sentiment.csv
│   │   │   ├── AFKS_price_vs_sentiment.png
│   │   │   ├── AFKS_sentiment_distribution.png
│   │   │   ├── AFKS_sentiment_dynamics.png
│   │   │   ├── AFKS_sentiment_features.csv
│   │   │   └── AFKS_topics.txt
│   │   └── tech_analysis
│   │       ├── AFKS_MACD.png
│   │       ├── AFKS_price_indicators.png
│   │       └── AFKS_tech_indicators.csv
...
│   ├── VTBR
│   │   ├── VTBR.parquet
│   │   ├── VTBR_all_news_processed.csv
│   │   ├── VTBR_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── VTBR_daily_events.csv
│   │   │   ├── VTBR_daily_sentiment.csv
│   │   │   ├── VTBR_event_distribution.png
│   │   │   ├── VTBR_ml_features.csv
│   │   │   ├── VTBR_news_with_events.csv
│   │   │   ├── VTBR_news_with_sentiment.csv
│   │   │   ├── VTBR_price_vs_sentiment.png
│   │   │   ├── VTBR_sentiment_distribution.png
│   │   │   ├── VTBR_sentiment_dynamics.png
│   │   │   ├── VTBR_sentiment_features.csv
│   │   │   └── VTBR_topics.txt
│   │   └── tech_analysis
│   │       ├── VTBR_MACD.png
│   │       ├── VTBR_price_indicators.png
│   │       └── VTBR_tech_indicators.csv
│   ├── bonds_csv
│   │   ├── bond_search_2023-12-01.csv
│   │   ├── bond_search_2024-03-26.csv
│   │   ├── bond_search_2024-04-10.csv
│   │   ├── bond_search_2024-05-24.csv
│   │   ├── bond_search_2024-08-09.csv
│   │   ├── bond_search_2024-08-10.csv
│   │   ├── bond_search_2024-08-12.csv
│   │   ├── bond_search_2024-09-04.csv
│   │   ├── bond_search_2024-09-13.csv
│   │   ├── bond_search_2024-10-30.csv
│   │   ├── bond_search_2024-11-26.csv
│   │   ├── bond_search_2024-12-04.csv
│   │   ├── bond_search_2025-01-28.csv
│   │   ├── bond_search_2025-02-10.csv
│   │   ├── bond_search_2025-02-14.csv
│   │   └── bond_search_2025-04-22.csv
...
├── raw_data
│   ├── AFKS
│   │   ├── BBG004S68614_2024
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240102.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240103.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240104.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240105.csv
...

├── status.json
├── telegram_news
│   ├── AFKS_telegram_news_20250422.csv
│   ├── ALRS_telegram_news_20250422.csv
│   ├── GAZP_telegram_news_20250422.csv
│   ├── GMKN_telegram_news_20250422.csv
│   ├── LKOH_telegram_news_20250422.csv
│   ├── MAGN_telegram_news_20250422.csv
│   ├── MTSS_telegram_news_20250422.csv
│   ├── MVID_telegram_news_20250422.csv
│   ├── NVTK_telegram_news_20250422.csv
│   ├── PHOR_telegram_news_20250422.csv
│   ├── ROSN_telegram_news_20250422.csv
│   ├── RUAL_telegram_news_20250422.csv
│   ├── SBER_telegram_news_20250422.csv
│   ├── SIBN_telegram_news_20250422.csv
│   ├── SNGS_telegram_news_20250422.csv
│   ├── TATN_telegram_news_20250422.csv
│   ├── VTBR_telegram_news_20250422.csv
│   └── telegram_headlines_20250422_160151.csv
└── tickers.csv) - конец папки data
├── ipynbs
│   ├── market_data.log
│   └── overall_data_pipeline.ipynb - тут в виде ноутбука (потому будет py файлом) расписан полный пайплайн сбора тикеров, тех анализа, фундаментального анализа, извлечения фичей из новостей и получение объединенного датасета со всеми фичами для всех тикеров (df.csv)
├── news_pipeline_config.json
├── pys
│   ├── __pycache__
│   │   ├── event_detector.cpython-313.pyc
│   │   ├── news_feature_extractor.cpython-313.pyc
│   │   ├── news_integration.cpython-313.pyc
│   │   ├── news_preprocessor.cpython-313.pyc
│   │   ├── news_visualizer.cpython-313.pyc
│   │   └── sentiment_analysis.cpython-313.pyc
│   ├── data_integration.py
│   ├── event_detector.py
│   ├── fundamental_data.py
│   ├── market_data.py
│   ├── news_feature_extractor.py
│   ├── news_integration.py
│   ├── news_pipeline.py
│   ├── news_preprocessor.py
│   ├── news_visualizer.py
│   ├── portfolio_optimizer.py
│   ├── run_with_config.py
│   ├── sentiment_analysis.py
│   ├── tech_analysis.py
│   └── technical_indicators.py
└── telegram_session.session

2) я еще вообще никак не работал с данными bonds, так что с ними что-то надо сделать. 

у меня есть 7 дней на выполнение всего курсового проекта, так что сейчас от тебя мне нужно много кода. предложи в соответствии с планом, что мне стоит сейчас сделать -->

In [149]:
#####

import signal_generator
importlib.reload(signal_generator)
from signal_generator import SignalGenerator

#####

import portfolio_optimizer
importlib.reload(portfolio_optimizer)
from portfolio_optimizer import PortfolioOptimizer

#####

import portfolio_optimizer
importlib.reload(portfolio_optimizer)
from portfolio_optimizer import PortfolioOptimizer

In [150]:
signal_gen = SignalGenerator(
    input_file='/Users/aeshef/Documents/GitHub/kursach/data/df.csv',
    weight_tech=0.5,
    weight_sentiment=0.3,
    weight_fundamental=0.2
)

signals_df = signal_gen.run_pipeline(
    output_file='/Users/aeshef/Documents/GitHub/kursach/data/signals.csv',
    output_dir='signal_visualizations'
)

2025-04-22 21:21:40,740 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-22 21:21:40 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-22 21:21:40,743 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-22 21:21:40 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-22 21:21:40,981 - signal_generator - INFO - Загружено 8049 строк данных
2025-04-22 21:21:40 - signal_generator - INFO - Загружено 8049 строк данных
2025-04-22 21:21:40,982 - signal_generator - INFO - Расчет композитного скора
2025-04-22 21:21:40 - signal_generator - INFO - Расчет композитного скора
2025-04-22 21:21:40,990 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-22 21:21:40 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-22 21:21:40,994 - signal_generator - INFO - Сентимен

In [151]:
bonds_results = run_pipeline_bonds(
    bonds_dir="/Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/csv",
    visualize=True,
    select_top_n=5
)
risk_free_rate = bonds_results['risk_free_rate']

2025-04-22 21:21:56 - bonds_pipeline - INFO - Запуск пайплайна обработки облигаций в 2025-04-22 21:21:56
2025-04-22 21:21:56 - bonds_pipeline - INFO - Загружены предыдущие рекомендации от 2025-04-22
2025-04-22 21:21:56 - bonds_pipeline - INFO - Начинаем загрузку и обработку данных по облигациям из /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/csv
2025-04-22 21:21:56 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2023-01-18.csv с 24 облигациями от 2023-01-18
2025-04-22 21:21:56 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2023-02-09.csv с 67 облигациями от 2023-02-09
2025-04-22 21:21:56 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2023-06-14.csv с 17 облигациями от 2023-06-14
2025-04-22 21:21:56 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2023-06-27.csv с 23 облигациями от 2023-06-27
2025-04-22 21:21:56 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2023-06-28.csv с 23 облигациями от 2023-06-28
20


Топ рекомендуемых облигаций:
    security_code            full_name  yield  duration_years  \
771  RU000A10ATB6  Интерскол КЛС БО-03  33.13        0.294167   
778  RU000A1070X5        АБЗ-1 001P-05  33.18        0.752500   
777  RU000A106SF2  Интерлизинг 001Р-06  31.96        0.655000   
745  RU000A106540    МВ ФИНАНС 001Р-04  31.86        0.910833   
774  RU000A1077X0  Интерлизинг 001Р-07  31.14        0.710833   

     risk_adjusted_yield  bond_score  weight  
771           112.623229    1.557725  0.3881  
778            44.093023    1.526310  0.1517  
777            48.793893    1.389869  0.1743  
745            34.978957    1.290518  0.1253  
774            43.807737    1.248066  0.1606  
Обработано 46 записей облигаций
Текущая безрисковая ставка: 26.42%

Топ рекомендуемых облигаций:
    security_code            full_name  yield  duration_years  \
771  RU000A10ATB6  Интерскол КЛС БО-03  33.13        0.294167   
778  RU000A1070X5        АБЗ-1 001P-05  33.18        0.752500   
777  

In [152]:
optimizer = PortfolioOptimizer(
    risk_free_rate=risk_free_rate,
    min_rf_allocation=0.3,
    max_rf_allocation=0.4
)
portfolio = optimizer.run_pipeline(
    df=signals_df,
    output_dir='/Users/aeshef/Documents/GitHub/kursach/data'
)

2025-04-22 21:21:57,685 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-22 21:21:57 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-22 21:21:57,687 - portfolio_optimizer - INFO - Использован предоставленный DataFrame с 8049 строками
2025-04-22 21:21:57 - portfolio_optimizer - INFO - Использован предоставленный DataFrame с 8049 строками
2025-04-22 21:21:57,688 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-22 21:21:57 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-22 21:21:57,692 - portfolio_optimizer - INFO - Отфильтровано 7676 строк по shortlist и сигналам
2025-04-22 21:21:57 - portfolio_optimizer - INFO - Отфильтровано 7676 строк по shortlist и сигналам
2025-04-22 21:21:57,717 - portfolio_optimizer - INFO - Рассчитаны доходности для 17 тикеров
2025-04-22 21:21:57 - portfolio_optimizer - INFO - Рассчитаны доходности для 17 тикеров
2025-04-22 21:21:57,71

In [153]:
backtester = Backtester(
    portfolio_weights=portfolio['weights'],
    start_date='2024-01-01',
    end_date='2025-04-22'
)
backtest_results = backtester.run_pipeline(
    df=signals_df,
    output_dir='portfolio_results/backtest'
)


NameError: name 'Backtester' is not defined